<a href="https://colab.research.google.com/github/firdaaacy/Praktikum-PPM/blob/main/Praktikum%207.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Praktikum Pengantar Pembelajaran Mesin


---
## Bab 7. Support Vector Machine (SVM) Lanjutan


### 1) Import Data

Unduh dataset yang akan digunakan pada praktikum kali ini. Anda dapat menggunakan aplikasi wget untuk mendowload dataset dan menyimpannya dalam Google Colab. Jalankan cell di bawah ini untuk mengunduh dataset

In [96]:
#mengunduh dataset iris dari link yang sudah diberikan
! wget https://dataset-ppm.s3.amazonaws.com/iris.csv

--2021-07-10 12:35:17--  https://dataset-ppm.s3.amazonaws.com/iris.csv
Resolving dataset-ppm.s3.amazonaws.com (dataset-ppm.s3.amazonaws.com)... 52.216.138.243
Connecting to dataset-ppm.s3.amazonaws.com (dataset-ppm.s3.amazonaws.com)|52.216.138.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4609 (4.5K) [text/csv]
Saving to: ‘iris.csv.5’

iris.csv.5          100%[===================>]   4.50K  --.-KB/s    in 0s      

2021-07-10 12:35:17 (209 MB/s) - ‘iris.csv.5’ saved [4609/4609]



Setelah dataset berhasil diunduh, langkah berikutnya adalah membaca dataset dengan memanfaatkan fungsi **readcsv** dari library pandas. Lakukan pembacaan berkas csv ke dalam dataframe dengan nama **data** menggunakan fungsi **readcsv**. Jangan lupa untuk melakukan import library pandas terlebih dahulu


In [97]:
#mengimpor library pandas untuk data analitis
import pandas as pd
#mengimpor library numpy untuk array
import numpy as np
#membaca data iris dengan library pandas dengan method read csv
data = pd.read_csv('iris.csv')



Cek isi dataset Anda dengan menggunakan perintah **head()**

In [98]:
#melihat 5 data teratas dengan method .head()
data.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


## 2) Membagi data menjadi data latih dan data uji

Metode pembelajaran mesin memerlukan dua jenis data :


1.   Data latih : Digunakan untuk proses training metode klasifikasi
2.   Data uji : Digunakan untuk proses evaluasi metode klasifikasi

Data uji dan data latih perlu dibuat terpisah (mutualy exclusive) agar hasil evaluasi lebih akurat.

Data uji dan data latih dapat dibuat dengan cara membagi dataset dengan rasio tertentu, misalnya 80% data latih dan 20% data uji.

Library Scikit-learn memiliki fungsi [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) pada modul **model_selection** untuk membagi dataset menjadi data latih dan data uji. Bagilah dataset anda menjadi dua, yaitu **data_latih** dan **data_uji**.


In [99]:
#melakukan import sebagian library yang akan digunakan
#pada kasus  ini hanya mengimport train_test_split dari modul sklearn.model_selection
from sklearn.model_selection import train_test_split
#melakukan pembagian dataset dengan pembagian 20% data uji dan 80% data latih
data_latih, data_uji = train_test_split(data, test_size = 0.2, random_state = 10)

Tampilkan banyaknya data pada **data_latih** dan **data_uji**. Seharusnya **data_latih** terdiri dari 120 data, dan **data_uji** terdiri dari 30 data

In [100]:
#menampilkan jumlah baris data uji yang sudah dibagi 20% dari dataset
print(data_uji.shape[0])
#menampilkan jumlah baris data latih yang sudah dibagi 80% dari dataset
print(data_latih.shape[0])

30
120


Pisahkan label/kelas dari data uji menjadi sebuah variabel bernama **label_uji**

In [101]:
#melakukan pemisahan antara fitur dan label dengan melakukan pop pada kolom spesies pada label uji
label_uji = data_uji.pop('species')

## 3) Pembentukan data latih one-vs-rest

Metode one-vs-rest memerlukan tiga jenis data latih yang diperlukan untuk melatih tiga SVM yang berbeda pada dataset Iris. Fungsi **buat_trainingset** digunakan untuk membentuk tiga dataset tersebut.

In [102]:
# membuat fungsi untuk memisahkan dataset berdasarkan kelas
def buat_trainingset (dataset):
  #membuat dictionary kosong
  trainingset = {}
  # mengambil kelas dalam dataset
  kolom_kelas = dataset.columns[-1]
  # mengambil kelas yang unik saja (menghilangkan duplikat)
  list_kelas = dataset[kolom_kelas].unique()
  # print(kolom_kelas)
  # print(list_kelas)
  #perulangan untuk setiap kelas
  for kelas in list_kelas:
    # melakukan penggandaan dataset 
    data_temp = dataset.copy(deep=True)
    # mengganti kelas dengan 1 ketika nilai species sama dengan kelas
    data_temp[kolom_kelas]=data_temp[kolom_kelas].map({kelas:1})
    # mengganti semua species yang tidak sama dengan kelas dengan -1
    data_temp[kolom_kelas]=data_temp[kolom_kelas].fillna(-1)
    # menyipan pada dictionary sesuai dengan nama kelas
    trainingset[kelas]=data_temp
  # mengembalikan trainingset yang sudah terpisah
  return trainingset

Gunakan fungsi **buat_trainingset** untuk membentuk data latih dengan nama variabel **trainingset** yang akan digunakan pada proses training. 

In [103]:
# memisahkan species menggunakan method yang sudah dibuat
trainingset = buat_trainingset(data_latih)

Tampilkan isi **trainingset** agar Anda dapat memahami struktur dari variabel tersebut.

In [104]:
# menampilkan dataset yang sudah dipisahkan pada variable trainingset
trainingset

{'Iris-setosa':      sepal_length  sepal_width  petal_length  petal_width  species
 58            6.6          2.9           4.6          1.3     -1.0
 97            6.2          2.9           4.3          1.3     -1.0
 129           7.2          3.0           5.8          1.6     -1.0
 114           5.8          2.8           5.1          2.4     -1.0
 146           6.3          2.5           5.0          1.9     -1.0
 ..            ...          ...           ...          ...      ...
 113           5.7          2.5           5.0          2.0     -1.0
 64            5.6          2.9           3.6          1.3     -1.0
 15            5.7          4.4           1.5          0.4      1.0
 125           7.2          3.2           6.0          1.8     -1.0
 9             4.9          3.1           1.5          0.1      1.0
 
 [120 rows x 5 columns],
 'Iris-versicolor':      sepal_length  sepal_width  petal_length  petal_width  species
 58            6.6          2.9           4.6          

## 4) Pembentukan SVM Biner

Tujuan dari algoritma SVM adalah meminimalkan nilai *cost function*. Penghitungan nilai minimal dapat dapat dilakukan dengan menghitung nilai gradien dari *cost function* terlebih dahulu. Fungsi di bawah ini berguna untuk menghitung nilai gradien cost function

In [105]:
# menghitung cost gradien menggunakan fungsi 
def hitung_cost_gradient(W,X,Y,regularization):
  #menghitung jarak dengan mengurangkan 1 dengan y * dot product x dan w
  jarak = 1 - (Y* np.dot(X,W))
  #menbuat array 0 seukuran dengan panjang w
  dw = np.zeros(len(W))
  #kondisi dimana jarak < -1
  if max(0,jarak)==0:
    #nilai di maka nilai W
    di= W
  #jika tidak memenuhi kondisi maka kondisi else akan dieksekusi
  else:
    #nilai di akan diset dari W - (regularization*Y*X)
    di = W - (regularization * Y*X)
  #nilai di akan disimpan dalam persamaan
  dw += di
  #mengembalikan nilai persamaan
  return dw

Terdapat beberapa cara untuk meminimalkan nilai *cost function*, salah satunya menggunakan Stochastic Gradient Descent (SGD) untuk melakukan minimasi. Minimasi *cost function* merupakan inti dari algoritma SVM. Fungsi di bawah ini merupakan implementasi algoritma SGD 

In [106]:
# mengimport kelas shuffle dari library sklearn utils
from sklearn.utils import shuffle
#membuat fungsi SGD untuk menghitung bobot
def sgd(data_latih,label_latih,learning_rate = 0.000001,max_epoch=1000,regularization=10000):
  #mengubah data latih yang berbentuk dataframe menjadi array mengunakan numpy
  data_latih = data_latih.to_numpy()
  #mengubah label latih yang berbentuk dataframe menjadi array mengunakan numpy
  label_latih = label_latih.to_numpy()
  #membuat variabel bobot yang berisi aray kosong yang ukurannya sama dengan banyak kolom
  bobot = np.zeros(data_latih.shape[1])
  #membuat perulangan untuk menghitung bobot
  for epoch in range(1,max_epoch):
    #melakukan shuffling nilai X dan Y agar tidak terjadi pola perhitungan bobot yang sama
    X,Y =shuffle(data_latih,label_latih,random_state=101)
    for index,x in enumerate(X):
      #menghitung delta dengan fungsi hitung cost gradient
      delta=hitung_cost_gradient(bobot,x,Y[index],regularization)
      #menentukan nilai bobot dengan mengurangi bobot dengan learning rate * delta
      bobot = bobot - (learning_rate * delta)
  #mengembalikan nilai bobot
  return bobot

## 5) Proses Training

Proses training dilakukan dengan memanggil fungsi **sgd** berulang kali sesuai banyaknya kelas yang ada pada data. Dengan demikian, proses training menghasilkan bobot sebanyak kelas yang ada pada dataset. Buatlah fungsi bernama **training** yang digunakan untuk melakukan proses training one-vs-rest

In [107]:
# membuat fungsi training untuk membuat model
def training(trainingset):
  # mengambil key pada trainingset
  list_kelas = trainingset.keys()
  # membuat dictionary kosong
  w = {}
  # melakukan perulangan untuk setiap kelas
  for kelas in list_kelas:
    # mengambil data latih sesuai dengan kelas
    data_latih = trainingset[kelas]
    # mengambil label untuk datalatih dengan fungsi pop
    label_latih = data_latih.pop(data_latih.columns[-1])
    # menyimpang masing masing persamaan sgd pada dictionary w
    w[kelas] = sgd(data_latih,label_latih)
  # mengembalikan nilai weight
  return w

Lakukan proses training dengan memanggil fungsi **training** dan menempatkan hasilnya pada variabel **W**

In [108]:
#menghitung nilai bobot dengan fungsi training
W = training(trainingset)

Tampilkan isi variabel **W**

In [109]:
#mencetak nilai W
print(W)

{'Iris-versicolor': array([ 0.96299733, -1.91811826,  0.4773442 , -2.02018242]), 'Iris-virginica': array([-3.77860219, -4.07274402,  5.21307293,  5.91165497]), 'Iris-setosa': array([ 0.22554941,  0.70766782, -1.1203483 , -0.54084396])}


## 6) Proses *testing* biner
Proses testing dilakukan dengan menghitung nilai [*dot product*](https://en.wikipedia.org/wiki/Dot_product) antara bobot hasil training dengan data uji. Kelas data ditentukan berdasarkan tanda (positif atau negatif) dari hasil dot product tersebut. Fungsi berikut mengimplementasikan proses testing

In [110]:
#membuat fungsi testing 
def testing(W,data_uji):
  #membuat array kosong yang akan digunakan menyimpan nilai prediksi
  prediksi = np.array([])
  #melakukan perulangan untuk menguji tiap baris data uji
  for i in range(data_uji.shape[0]):
    #melakukan perhitungan dot product antara data uji dengan matriks W
    y_prediksi = np.sign(np.dot(W,data_uji.to_numpy()[i]))
    #menambahkan nilai prediksi ke array prediksi
    prediksi = np.append(prediksi,y_prediksi)
  # mengembalikan nilai prediksi secara keseluruhan
  return prediksi

## TUGAS
Pada tugas kali ini Anda mendefinisikan proses testing pada metode one-vs-rest. Proses testing pada metode one-vs-rest dilakukan dengan memanggil proses testing biner untuk setiap **value** pada dictionary **W**. Kelas pada sebuah data latih adalah **key** pada dictionary **W** yang memiliki nilai prediksi **1**. Lengkapi fungsi **testing_onevsrest** di bawah ini. Output dari fungsi tersebut adalah list nama kelas hasil prediksi.

In [111]:
list_kelas = W.keys()
list_kelas

dict_keys(['Iris-versicolor', 'Iris-virginica', 'Iris-setosa'])

In [112]:
def testing_onevsrest(W,data_uji):
  # membuat variabel dengan isi jenis kelas pada keys W
  list_kelas = W.keys()
  # membuat dataframe yang memiliki kolom iris-virginica, iris-setosa, dan iris-versicolor
  hasil = pd.DataFrame(columns = list_kelas)
  # perulangan untuk setiap kelas
  for kelas in list_kelas:
    # menambahkan nilai prediksi sesuai dengan kolom kelas
    hasil[kelas] = testing(W[kelas],data_uji)
  # menentukan kelas_prediksi sesuai dengan nilai maksimum pada setiap baris
  temp_kelas_prediksi = hasil.idxmax(axis = 1)
  # mengubah type data dataframe ke numpy
  kelas_prediksi = temp_kelas_prediksi.to_numpy()
  # mengembalikan hasil prediksi
  return kelas_prediksi

In [113]:
# melakukan prediksi dengan pemanggilan fungsi testing_onevsrest
prediksi = testing_onevsrest(W,data_uji)
# menampilkan hasil prediksi
prediksi

array(['Iris-versicolor', 'Iris-virginica', 'Iris-setosa',
       'Iris-versicolor', 'Iris-setosa', 'Iris-versicolor',
       'Iris-versicolor', 'Iris-versicolor', 'Iris-setosa',
       'Iris-versicolor', 'Iris-versicolor', 'Iris-virginica',
       'Iris-versicolor', 'Iris-setosa', 'Iris-setosa', 'Iris-virginica',
       'Iris-versicolor', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
       'Iris-virginica', 'Iris-virginica', 'Iris-virginica',
       'Iris-setosa', 'Iris-versicolor', 'Iris-setosa', 'Iris-versicolor',
       'Iris-versicolor', 'Iris-versicolor', 'Iris-virginica'],
      dtype=object)

Berapa banyak data latih yang berhasil diprediksi dengan benar?

In [114]:
#menampilkan jumlah prediksi yang sama dengan label sebenarnya
prediksiBenar = sum(prediksi==label_uji)
#menampilkan jumlah data sebenarnya
jumlahSebenarnya = label_uji.shape[0]
#menampilkan nilai prediksi yang benar
print("Prediksi benar =", prediksiBenar)
# menampilkan jumlah data sebenarnya
print("Jumlah label sebenarnya =", jumlahSebenarnya)
# menghitung akurasi dengan membandingkan nilai benar dengan jumlah 
print("Akurasi = ", (prediksiBenar/jumlahSebenarnya))

Prediksi benar = 30
Jumlah label sebenarnya = 30
Akurasi =  1.0


Terdapat 29 data latih yang berhasil diprediksi dengan benar dari total 30 data uji sehingga akurasi yang didapatkan adalah 0.96